In [2]:
import wandb
import os



Saved in 

`onpolicy/results/GraphMPE/navigation_graph/rmappo/informarl/wandb`

to run evaluation script, use:

```
xvfb-run -a -s "-screen 0 1400x900x24" python -u onpolicy/scripts/eval_mpe.py --model_dir onpolicy/results/GraphMPE/navigation_graph/rmappo/informarl/wandb/run-20250312_024502-j08n9m0q/files

python onpolicy/scripts/eval_mpe.py --model_dir onpolicy/trained_models/informarl_3agents_target/files
```



In [2]:
import pyglet
pyglet.window.Window()

NoSuchDisplayException: Cannot connect to "None"

t

- 3 agents at 
> `onpolicy/results/GraphMPE/navigation_graph/rmappo/informarl/wandb/run-20250312_024502-j08n9m0q/files`

In [3]:
import torch
print(torch.__version__)

2.5.1
